# **Initialisation**
Initialisation de la base de données spark

In [231]:
import os
import findspark

findspark.init()

import spark


In [232]:
import configparser

config = configparser.ConfigParser()

In [233]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").appName("Import").getOrCreate()

# **Extraction**
Premier regard sur la base de données 

In [234]:
env = "macOS"

if env == "macOS" :
      df = spark.read.option("multiline","true").json("/Users/nicolassigal/Desktop/Scolaire/ENSTA/3A/ASI322/PROJECT/CVE_Analysis/nvdcve-1.1-2022.json")
if env == "Linux" :
      df = spark.read.option("multiline","true").json("/home/marins/3A-ENSTA/Big data project/nvdcve-1.1-2022.json")


print("Schema initial : ")
df.printSchema()

df = df.select(explode(col("CVE_Items")))

Schema initial : 
root
 |-- CVE_Items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- configurations: struct (nullable = true)
 |    |    |    |-- CVE_data_version: string (nullable = true)
 |    |    |    |-- nodes: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |-- children: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |    |    |-- cpe_match: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |    |-- cpe23Uri: string (nullable = true)
 |    |    |    |    |    |    |    |    |    |-- cpe_name: array (nullable = true)
 |    |    |    |    |    |    |   

Extraction des données pertinents via select(). 

In [235]:
df = df.select("col.cve.CVE_data_meta.*","col.cve.description.description_data.lang","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")

print("Columns : ",df.columns)
#df.printSchema()

Columns :  ['ASSIGNER', 'ID', 'lang', 'attackComplexity', 'attackVector', 'availabilityImpact', 'baseScore', 'baseSeverity', 'confidentialityImpact', 'integrityImpact', 'privilegesRequired', 'scope', 'userInteraction']


Affichage des premières lignes et du nombre de samples à disposition pour se faire une idée.

In [236]:
print("Number of available rows : ", df.count())
df.show(5)

Number of available rows :  17677
+--------------------+-------------+----+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|            ASSIGNER|           ID|lang|attackComplexity|attackVector|availabilityImpact|baseScore|baseSeverity|confidentialityImpact|integrityImpact|privilegesRequired|    scope|userInteraction|
+--------------------+-------------+----+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|    secure@intel.com|CVE-2022-0001|[en]|             LOW|       LOCAL|              NONE|      6.5|      MEDIUM|                 HIGH|           NONE|               LOW|  CHANGED|           NONE|
|    secure@intel.com|CVE-2022-0002|[en]|             LOW|       LOCAL|              NONE|      6.5|      MEDIUM|                 HIGH|           NONE|               LOW|  CHANGE

Essai de la fonction tail pour récupérer des données. Pas idéal. La fonction "collect()" semble bien plus adaptée.

In [237]:
print(df.tail(1)) 

[Row(ASSIGNER='cve@mitre.org', ID='CVE-2022-45939', lang=['en'], attackComplexity=None, attackVector=None, availabilityImpact=None, baseScore=None, baseSeverity=None, confidentialityImpact=None, integrityImpact=None, privilegesRequired=None, scope=None, userInteraction=None)]


# Représentation graphique
Tentative de plot.

In [238]:
import pandas as pd
pandasDF = df.toPandas()
print(pandasDF.columns)

print(pandasDF.shape[0])

pandasDF = pandasDF[pandasDF['baseScore'].notna()]

print(pandasDF.shape[0])


pandasDF.head(3)

Index(['ASSIGNER', 'ID', 'lang', 'attackComplexity', 'attackVector',
       'availabilityImpact', 'baseScore', 'baseSeverity',
       'confidentialityImpact', 'integrityImpact', 'privilegesRequired',
       'scope', 'userInteraction'],
      dtype='object')
17677
17220


,ASSIGNER,ID,lang,attackComplexity,attackVector,availabilityImpact,baseScore,baseSeverity,confidentialityImpact,integrityImpact,privilegesRequired,scope,userInteraction
0,secure@intel.com,CVE-2022-0001,[en],LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
1,secure@intel.com,CVE-2022-0002,[en],LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
2,secure@intel.com,CVE-2022-0004,[en],LOW,PHYSICAL,HIGH,6.8,MEDIUM,HIGH,HIGH,NONE,UNCHANGED,NONE


In [239]:
target_x_column = "lang"
target_y_column = "baseScore"
targetx = pandasDF[target_x_column]
targety = pandasDF[target_y_column]

categorical_columns = ["attackComplexity",
                        "attackVector",
                        "availabilityImpact",
                        "confidentialityImpact",
                        "integrityImpact",
                        "privilegesRequired",
                        "scope",
                        "userInteraction"]

data = pandasDF[categorical_columns]


Affichage des types des différentes colonnes

In [240]:
targetx.dtypes

dtype('O')

In [241]:
data.dtypes

attackComplexity         object
attackVector             object
availabilityImpact       object
confidentialityImpact    object
integrityImpact          object
privilegesRequired       object
scope                    object
userInteraction          object
dtype: object

In [246]:
import numpy as np
import matplotlib.pyplot as plt

#pd.set_option("display.max_rows", None)

#i = 0
#print(len(targetx))
L = []
for i in range(0,100) :
      L.append(targetx[i][0])

plt.plot(L,targety[0:100],"bo")

#pandasDF.loc[116]

KeyError: 116

In [243]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Onehot ==> Linéaire
# Ordinal = Treebased

model = make_pipeline(OneHotEncoder(),LinearRegression())
model

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(categories='auto', drop=None,
                               dtype=<class 'numpy.float64'>,
                               handle_unknown='error', sparse=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [244]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv = 10)


/Users/nicolassigal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
/Users/nicolassigal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
/Users/nicolassigal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: setting an array element with a sequence.

  FitFailedWarning)
/Users/nicolassigal/opt/anaconda3/lib/python3.7/site-packages/sklea

In [245]:
cv_results["test_score"].mean()

nan